In [10]:
suppressMessages(library(rwwa))

# Check effect of choice of VPD calculation on fitted trend in HDWI

## GEV

In [70]:
gmst <- read.table("ts/gmst.txt", col.names = c("year", "gmst"))

cov_2025 <- gmst[gmst$year == 2025,"gmst",drop = F]
cov_pi <- cov_2025 - 1.3

rnm = "korea"
for(fnm in list.files("ts", pattern = paste0("hdwi.+ERA5land_",rnm), full.names = T)) {
    res_fnm <- gsub(".dat", "_gev.csv", gsub("ts/korea-fires", "res/res-obs", fnm))
    
    if(!file.exists(res_fnm)) {
        ts <- load_ts(fnm, col.names = c("year", "hdwi5x"))
        df <- merge(gmst, ts)
        mdl <- fit_ns("gev", "shift", data = df, varnm = "hdwi5x", covnm = c("gmst"), lower = F)
        boot_res <- boot_ci(mdl, cov_f = cov_2025, cov_cf = cov_pi)
        write.csv(boot_res, res_fnm)
    }
}

## lognormal

In [72]:
gmst <- read.table("ts/gmst.txt", col.names = c("year", "gmst"))

cov_2025 <- gmst[gmst$year == 2025,"gmst",drop = F]
cov_pi <- cov_2025 - 1.3

rnm = "korea"
for(fnm in list.files("ts", pattern = paste0("hdwi.+ERA5land_",rnm), full.names = T)) {
    res_fnm <- gsub(".dat", "_lnorm.csv", gsub("ts/korea-fires", "res/res-obs", fnm))
    
    if(!file.exists(res_fnm)) {
        ts <- load_ts(fnm, col.names = c("year", "hdwi5x"))
        ts$hdwi5x <- log(ts$hdwi5x)
        df <- merge(gmst, ts)
        mdl <- fit_ns("norm", "shift", data = df, varnm = "hdwi5x", covnm = c("gmst"), lower = F)
        boot_res <- boot_ci(mdl, cov_f = cov_2025, cov_cf = cov_pi)
        write.csv(boot_res, res_fnm)
    }
}

In [ ]:
prep_window(c(1,2))

plot(density(ts$hdwi5x), ylim = c(0,2))
mdl <- strip(fevd(ts$hdwi5x))
lines(x <- seq(0,3,0.01), devd(x, loc = mdl["location"], shape = mdl["shape"], scale = mdl["scale"]), col = "blue")

plot(density(log(ts$hdwi5x)), ylim = c(0,2))
lines(x <- seq(-0.5,1.5,0.01), dnorm(x, mean(log(ts$hdwi5x)), sd(log(ts$hdwi5x))), col = "blue")